In [25]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.ndimage
import os
import cv2
from tqdm import tqdm
from utils import get_2D_projections
from datetime import datetime
import traceback

%env SITK_SHOW_COMMAND 'C:\ProgramData\slicer.org\Slicer 5.4.0\Slicer'

env: SITK_SHOW_COMMAND='C:\ProgramData\slicer.org\Slicer 5.4.0\Slicer'


In [18]:
source_path_wd = "/media/andres/T7 Shield1/U-CAN-Lymfom_A"
source_path_bd = "/media/andres/T7 Shield/ucan_lymfom"

selection_dataframe = os.path.join(source_path_bd, "Finalized_dataset.xlsx" )

final_projections_bd = os.path.join(source_path_bd, 'Selected_for_UCAN_project/2dprojections')
final_projections_bd

'/media/andres/T7 Shield/ucan_lymfom/Selected_for_UCAN_project/2dprojections'

In [15]:
master_bd = pd.read_excel(selection_dataframe)
try:
    drop_col_list = [col for col in master_bd.columns if 'unnamed' in col.lower()]
    master_bd.drop(columns=drop_col_list, inplace=True)
except:
    pass
print(master_bd.shape)
master_bd.head(2)

(2264, 42)


,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,dicom_img,patient_id,patient_age,...,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX000...,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_...,ASPTCTX0001,lpr385705046400,20140313,1.2.840.113619.2.55.3.4240671514.332.139417595...,lpr385705046400,33.0,...,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX000...,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D...,ASPTCTX0001,lpr385705046400,20140313,1.2.840.113619.2.131.4240671514.1394722651.463822,lpr385705046400,33.0,...,PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [21]:
#Global Vars

MODA='PET' # PET CT
TYPE='LT'  # AT,LT,B,N
PTYPE='max' # max mean

In [51]:
def compute_suv(vol_img, PatientWeight, AcquisitionTime , RadiopharmaceuticalStartTime, RadionuclideHalfLife, RadionuclideTotalDose):
    
    estimated = False

    raw = sitk.GetArrayFromImage(vol_img)    
    spacing = vol_img.GetSpacing()
    origin = vol_img.GetOrigin()
    direction = vol_img.GetDirection() 

    #raw,spacing,origin,direction = imread(image_file_list)
    
    
    try:
        weight_grams = float(PatientWeight)*1000
    except:
        #traceback.print_exc()
        weight_grams = 75000
        estimated = True
        
    try:
        # Get Scan time
        scantime = datetime.strptime(AcquisitionTime,'%H%M%S')
        # Start Time for the Radiopharmaceutical Injection
        injection_time = datetime.strptime(RadiopharmaceuticalStartTime,'%H%M%S')
        # Half Life for Radionuclide # seconds
        half_life = float(RadionuclideHalfLife) 
        # Total dose injected for Radionuclide
        injected_dose = float(RadionuclideTotalDose)
        # Calculate decay
        decay = np.exp(-np.log(2)*((scantime-injection_time).seconds)/half_life);
        # Calculate the dose decayed during procedure
        injected_dose_decay = injected_dose*decay; # in Bq        
    except:
        #traceback.print_exc()
        decay = np.exp(-np.log(2)*(1.75*3600)/6588); # 90 min waiting time, 15 min preparation
        injected_dose_decay = 420000000 * decay; # 420 MBq
        estimated = True
    
    # Calculate SUV # g/ml
    suv = raw*weight_grams/injected_dose_decay
    
    return suv, estimated, raw,spacing,origin,direction

In [52]:
exception_lst = []
new_size = [384, 384, 384]
new_spacing = [2, 2, 3]

if not os.path.exists(final_projections_bd):
    os.mkdir(final_projections_bd)



for index, row in master_bd.iterrows():
    if row['modality']=='PT':
        #vol_img = sitk.ReadImage(row['directory'])
        
        print(row['directory'])

        reader = sitk.ImageSeriesReader()
        dicom_names = reader.GetGDCMSeriesFileNames(row['directory'])
        reader.SetFileNames(dicom_names)
        vol_img = reader.Execute() 

        vol_img = sitk.Resample(vol_img, new_size, sitk.Transform(), sitk.sitkLinear,
                            vol_img.GetOrigin(), new_spacing, vol_img.GetDirection(), 0,
                            vol_img.GetPixelID())
        
        save_path = os.path.join(final_projections_bd, str(row['patient_directory']) + '_PET_resample_3Dimg')
        
        get_2D_projections(vol_img,MODA,ptype=PTYPE,angle=15,img_n=save_path)
        
        suv, estimated, raw,spacing,origin,direction = compute_suv(vol_img, PatientWeight=row['patient_weight'], AcquisitionTime=str(row['aquisition_dt'])+str(row['aquisition_time']) , RadiopharmaceuticalStartTime=row['radiopharmaceutical_start_time'], RadionuclideHalfLife=row['radionuclide_half_life'], RadionuclideTotalDose=row['radionuclide_total_dose'])
        suv_img = sitk.GetImageFromArray(suv)
        
        save_path= os.path.join(final_projections_bd, str(row['patient_directory']) + '_SUV_resample_3Dimg')

        get_2D_projections(suv_img,MODA,ptype=PTYPE,angle=15,img_n=save_path)
        break

/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm
65569.66159971479 0.0
22.718786471713308 0.0


In [ ]:
v_img = sitk.ReadImage(os.path.join(source_path_wd, 'Ashish_SampleResampling/pat1_PET_resample_3Dimg.nii.gz'))
if not os.path.exists(os.path.join(source_path_wd, 'Ashish_SampleResampling/2d projections')):
    os.mkdir(os.path.join(source_path_wd, 'Ashish_SampleResampling/2d projections'))

save_path = os.path.join(final_projections_bd, 'pat1_PET_resample_3Dimg')

#get_2D_projections(v_img,MODA,PTYPE,15,T_TYPE, img_n=save_path)
get_2D_projections(v_img,MODA,ptype=PTYPE,angle=15,img_n=save_path)
save_path= os.path.join(final_projections_bd, '/CT_sample3Dimg')

get_2D_projections(v_img,MODA,TYPE,PTYPE,15,img_n=save_path)